In [9]:
!pip install openai pinecone-client langchain gradio unstructured tiktoken pypdf
!pip install -U langchain-community
!pip install PyPDF2

import openai
import pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
import gradio as gr
from google.colab import files
from PyPDF2 import PdfReader

OPENAI_API_KEY = "your-openai-api-key"
PINECONE_API_KEY = "your-pinecone-api-key"
PINECONE_ENVIRONMENT = 'us-east-1'
PINECONE_INDEX = 'business-qabot'
OPENAI_LLM = 'text-davinci-003'

uploaded = files.upload()

def read_doc(uploaded_files):
    text = ""
    for file_name in uploaded_files.keys():
        with open(file_name, "r", encoding="utf-8") as f:
            text += f.read() + "\n"
    return text

def chunk_data(doc, chunk_size=1000):
    chunks = [doc[i:i + chunk_size] for i in range(0, len(doc), chunk_size)]
    return chunks

def initialize():
    openai.api_key = OPENAI_API_KEY
    pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

def store_embeddings(documents):
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    index = Pinecone.from_documents(documents, embeddings, index_name=PINECONE_INDEX)
    return index

def retrieve_answers(index, chain, query):
    result = index.similarity_search(query, k=3)
    return chain.run(input_documents=result, question=query)

def main(query):
    doc = read_doc(uploaded)
    documents = chunk_data(doc)
    initialize()
    index = store_embeddings(documents)
    llm = OpenAI(model_name=OPENAI_LLM, temperature=0.3)
    chain = load_qa_chain(llm, chain_type="stuff")
    answer = retrieve_answers(index, chain, query)
    return answer

qa_app = gr.Interface(
    fn=main,
    inputs=[gr.Textbox(label="Ask a Question", placeholder="What do you want to know?")],
    outputs=[gr.Textbox(label="Answer")],
    title="Business QA Bot - Retrieval Augmented Generation (RAG)",
    description="A RAG model using OpenAI and Pinecone to answer questions from uploaded documents."
)

qa_app.launch()


Saving business_overview.pdf to business_overview (4).pdf
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ebaeda41b869b1a1a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
